In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import math
import json
import requests

from utility import *

pd.set_option('max_rows', None)

**Get title data (MAKE SURE SPREADSHEET END IS CORRECT WHEN ADDING NEW VOTERS)**

In [2]:
# Authenticate
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
service = build('sheets', 'v4', credentials=gsheet_api_check(SCOPES))

# Pull titles data
start = 'R' # start column of voters
end = 'CP480' # end column, end row of voters + 1
SPREADSHEET_ID = '1Rx-jnDhhyaXdUDr37EVM0yuOLOOhBw1cFj5asIfSlfg'
RANGE_NAME = 'Data!A1:'+chr(ord(start)-2)+end[2:]

data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)

COMPLETE: Data copied


In [3]:
# Convert list of lists into numpy array
x = np.array(data)
length = max(map(len, data))
y = np.array([xi+[None]*(length-len(xi)) for xi in x])
#y=np.array([xi for xi in x])
titlesdf = y[1:-1,0]

df = pd.DataFrame(y[1:-1,1:], columns=y[0,1:], index=titlesdf)
df.head(5)

,Rank,Votes,Score,Esoteric Rank,Esoteric Score,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Neon Genesis Evangelion,1,56,1238.1,148,17.9,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,None
Serial Experiments Lain,2,46,987.0,180,12.5,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/oQOhIIZWKZpeQS6uZejS9rhaHWO.jpg,None
Cowboy Bebop,3,43,870.0,185,12.2,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,None
FLCL,4,33,700.0,220,8.6,Original,18,Gainax,2000-04-26,2018-10-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,None
Revolutionary Girl Utena,5,28,658.0,186,12.0,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,None


**Get individual votes**

In [4]:
RANGE_NAME = 'Data!'+start+'1:'+end
vote_data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)

COMPLETE: Data copied


In [5]:
# Convert list of lists into numpy array
vx = np.array(vote_data)
length = max(map(len, vote_data))
vy = np.array([xi+[None]*(length-len(xi)) for xi in vx])

# This will fail if sheet sorted with 0 vote entries at the bottom
vdf = pd.DataFrame(vy[1:-1,:],columns=vy[0,:],index=titlesdf)

# Make sparse matrix
vdf = vdf.fillna('')
vdf = vdf.replace('',np.nan)
vdf = vdf.astype(pd.SparseDtype("int", np.nan))
print('Density:', vdf.sparse.density)
vdf.head(2)

Density: 0.049828832255610495


,acchonburike,apdoesta,Araby_,baanp,Bagman,BananaSam,bluedelirium,CaptainBoomerang,Chapnik,chumlum,...,ThelMi,Toadfan64,trimonuter,Triturate,Vancyon,Whoa,WeirderisBetter,xo_lauren,YasashiiDia,yellsicklecell
Neon Genesis Evangelion,NaN,NaN,2,1,3,2.0,5,17,NaN,22,...,2.0,NaN,13,7,NaN,45,1,-1,3,1
Serial Experiments Lain,NaN,NaN,11,4,2,NaN,6,43,1.0,10,...,NaN,NaN,12,15,NaN,1,19,-1,9,17


**Make df with voter meta data**

In [6]:
# Number of ranked votes
ranked_votes = [max(max(vdf[vdf[col].notna()][col]),0) for col in vdf.columns]

# Number of unranked votes
unranked_votes = [vdf[vdf[col]==-1][col].count() for col in vdf.columns]

# Total votes
from operator import add
total_votes= list(map(add, ranked_votes, unranked_votes))

vm_index = ['ranked_votes','unranked_votes', 'total_votes']
vm_data = [ranked_votes, unranked_votes, total_votes]

vmdf = pd.DataFrame(vm_data,columns=vy[0,:],index=vm_index)
vmdf

,acchonburike,apdoesta,Araby_,baanp,Bagman,BananaSam,bluedelirium,CaptainBoomerang,Chapnik,chumlum,...,ThelMi,Toadfan64,trimonuter,Triturate,Vancyon,Whoa,WeirderisBetter,xo_lauren,YasashiiDia,yellsicklecell
ranked_votes,1,1,50,11,50,10,14,50,10,50,...,10,10,35,30,5,49,33,0,50,25
unranked_votes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13,0,0
total_votes,1,1,50,11,50,10,14,50,10,50,...,10,10,35,30,5,49,33,13,50,25


**Calculate scores and rankings**

In [7]:
# Count votes per title
counts = vdf.count(axis=1)
df['Votes'] = counts.iloc[:].values

# Modified Borda count
df['Score'] = 0
df['Esoteric Score'] = 0

most_votes = max(df['Votes'])
weight=7
inv_pop_weight = 0.15

for title in vdf.index:
    
    points=0
    esoteric=0
    votes = df.loc[title,'Votes'] 
    
    if votes < 1:
        df.loc[title,'Score'] = 0
        df.loc[title,'Esoteric Score'] = 0
        continue
    
    for voter in vdf.columns:
        
        total_votes = vmdf.loc['total_votes',voter]
        ranked_votes = vmdf.loc['ranked_votes',voter]
        rank = vdf.loc[title,voter]
        
        if rank>0: # Ranked votes
            points += total_votes + 1 - rank
            esoteric += 1 + (total_votes-1)*np.exp(-(rank-1)**2 / (2*weight**2))
            
        elif rank == -1: # Unranked votes
            points += 0.5 * (1+total_votes-ranked_votes)
            esoteric += 1
    
    df.loc[title,'Score'] = points
    
    esoteric *= 1 + most_votes * np.exp(-(votes-1)**2 / (2*(inv_pop_weight*most_votes)**2))
    esoteric /= 1 + most_votes
    df.loc[title,'Esoteric Score'] = esoteric
    
df['Score'] += 0.001*df['Votes'] # hacky way of sorting ties by votes

In [8]:
# Rankings
df['Rank'] = df['Score'].rank(ascending=False,method='min').astype(int)
df['Esoteric Rank'] = df['Esoteric Score'].rank(ascending=False,method='min').astype(int)

# Rounding
df['Score'] = df['Score'].round(1)
df['Esoteric Score'] = df['Esoteric Score'].round(1)

df.loc[df['Score']<1,'Votes'] = 0

df.sort_values(by=['Rank']).head(5)

,Rank,Votes,Score,Esoteric Rank,Esoteric Score,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Neon Genesis Evangelion,1,56,1238.1,148,17.9,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,None
Serial Experiments Lain,2,46,987.0,180,12.5,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/oQOhIIZWKZpeQS6uZejS9rhaHWO.jpg,None
Cowboy Bebop,3,43,870.0,185,12.2,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,None
FLCL,4,33,700.0,220,8.6,Original,18,Gainax,2000-04-26,2018-10-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,None
Revolutionary Girl Utena,5,28,658.0,186,12.0,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,None


**Make df with titles meta data**

In [9]:
df['AltTitle'].replace(np.nan,'',inplace=True)

In [10]:
url = 'https://graphql.anilist.co'

query = '''
query ($id: Int, $page: Int, $perPage: Int, $search: String) {
    Page (page: $page, perPage: $perPage) {
        pageInfo {
            total
            currentPage
            lastPage
            hasNextPage
            perPage
        }
        media (id: $id, search: $search, type: ANIME) {
            id
            source
            genres
            episodes
            title {
                romaji
            }


            studios {
              edges {
                id
                isMain
                node {
                  name
                }
              }
            }


        }
    }
}
'''

In [11]:
# Query Anilist

df['Genres'].astype(object) # to insert lists

def AniQuery(df,variables,title,query=query):
    response = requests.post(url, json={'query': query, 'variables': variables})
    parsed = json.loads(response.text)
    dat = parsed['data']['Page']['media'][0]
    #pretty_data = json.dumps(dat, indent=4, sort_keys=True)
    #print(pretty_data,'\n')

    df.loc[title,'Source'] = dat['source']
    if dat['studios']['edges']:
        df.loc[title,'Studio'] = list(filter(lambda x:x["isMain"]==True,dat['studios']['edges']))[0]['node']['name']
    df.at[title, 'Genres'] = dat['genres']

    df.loc[title, 'AniListID'] = dat['id'] 

for i, title in enumerate(titlesdf):

    if i % 50 == 0:
        print(i,title)

    variables = {'search': title}
    
    if (df.loc[title, 'Source'] in [None, ''] or
        df.loc[title, 'Genres'] in [None, ''] or
        df.loc[title, 'Studio'] in [None, ''] or
        df.loc[title, 'AniListID'] in [np.nan,None, '']):

        try:
            
            if df.loc[title, 'AniListID']:
                variables = {"id": df.loc[title, 'AniListID']}
            AniQuery(df,variables,title)

        except KeyError:
            try:
                variables = {'search': df.loc[title,'AltTitle']}
                AniQuery(df,variables,title)
            except KeyError:
                print('KeyError:',title)
                continue
        except IndexError:
            try:
                variables = {'search': df.loc[title,'AltTitle']}
                AniQuery(df,variables,title)
            except IndexError:
                print('IndexError:', title, df.loc[title,'AltTitle'])
                continue

            
df.head()

0 Neon Genesis Evangelion
50 Death Note
IndexError: Giant Robo 
100 Space☆Dandy
IndexError: The Gutsy Frog 
150 Assassination Classroom
200 Key the Metal Idol
250 JoJo's Bizarre Adventure (1993)
IndexError: Cat Soup Theater 
300 Re:Zero
IndexError: Saber Marionette R 
IndexError: Doraemon 
350 Occultic;Nine
400 Haré+Guu
IndexError: Haré+Guu 
IndexError: Protect from the Shadows! Guardian Ninja Mamoru
IndexError: Binchou-tan 
IndexError: Manyuu Hikenchou 
IndexError: Astro Fighter Sunred 
450 Beck
IndexError: Hori-san to Miyamura-kun 


,Rank,Votes,Score,Esoteric Rank,Esoteric Score,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Neon Genesis Evangelion,1,56,1238.1,148,17.9,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,
Serial Experiments Lain,2,46,987.0,180,12.5,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/oQOhIIZWKZpeQS6uZejS9rhaHWO.jpg,
Cowboy Bebop,3,43,870.0,185,12.2,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,
FLCL,4,33,700.0,220,8.6,Original,18,Gainax,2000-04-26,2018-10-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,
Revolutionary Girl Utena,5,28,658.0,186,12.0,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,


In [12]:
# Flatten genres list
separator = ', '
for title in titlesdf:
    if type(df.loc[title,'Genres']) == list:
        df.loc[title,'Genres'] = separator.join(df.loc[title,'Genres'])


# Capitalize Source
df['Source'] = df['Source'].str.replace('_',' ')
df['Source'] = df['Source'].str.title()
        
df.head(1000)

,Rank,Votes,Score,Esoteric Rank,Esoteric Score,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Neon Genesis Evangelion,1,56,1238.1,148,17.9,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,
Serial Experiments Lain,2,46,987.0,180,12.5,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/oQOhIIZWKZpeQS6uZejS9rhaHWO.jpg,
Cowboy Bebop,3,43,870.0,185,12.2,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,
FLCL,4,33,700.0,220,8.6,Original,18,Gainax,2000-04-26,2018-10-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,
Revolutionary Girl Utena,5,28,658.0,186,12.0,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,
The Tatami Galaxy,6,28,583.0,201,9.8,Novel,11,MADHOUSE,2010-04-23,2010-07-02,"Comedy, Mystery, Psychological, Romance",7785,36243,/3Ngmk2kNh6JTdqPwMdiZvDSdlGH.jpg,
Mushishi,7,18,476.0,68,43.7,Manga,46,Artland,2005-10-23,2014-12-21,"Adventure, Fantasy, Mystery, Slice of Life, Su...",457,26867,/ianYYQaLkksLGCfeisFNYJE7jIO.jpg,
Ping Pong the Animation,8,25,470.0,237,7.0,Manga,11,Tatsunoko Production,2014-04-11,2014-06-20,"Drama, Psychological, Sports",20607,60811,/fUoKvf8jYB9fXkm7AuKcGVRCt2H.jpg,
Welcome to the NHK,9,24,416.0,229,7.6,Light Novel,24,GONZO,2006-07-09,2006-12-17,"Comedy, Drama, Psychological, Romance, Slice o...",1210,42821,/4EHSyDH8Z7A3qUjfrSM3aLwNHGH.jpg,
Haibane Renmei,10,22,414.0,163,14.7,Manga,13,Radix,2002-10-09,2002-12-18,"Drama, Fantasy, Mystery, Psychological, Slice ...",387,34164,/G8PFY3uKO8Lpdfmyce4VrlxfXm.jpg,


In [13]:
# Query TMDb.org

f = open("tmdb_api_key.txt", "r")
api_key = f.read()[:-1]
f.close()

for i, title in enumerate(titlesdf):

    if i % 50 == 0:
        print(i,title)
    
    if (df.loc[title,'Episodes'] in [None, ''] or
        df.loc[title,'First Air Date'] in [None, ''] or
        df.loc[title,'Last Air Date'] in [None, ''] or 
        df.loc[title, 'IMGID'] in [None,''] or 
        df.loc[title, 'TMDbID'] in [None,'']):
        
        title_id = str(df.loc[title, 'TMDbID'])
        
        try:
            if title_id in [None,'','None']:
                
                r = requests.get('https://api.themoviedb.org/3/search/tv?api_key='+api_key+'&query='+title)
                parsed = json.loads(r.text)  
                
                # Make sure TMDb genre contains animation (possibly: origin country is JP)
                for j, res in enumerate(parsed['results']):
                    if 16 in res['genre_ids']: # TMDb genre ID for animation = 16
                        title_id = str(res['id'])
                        df.loc[title, 'TMDbID'] = title_id
                        break

                if title_id in [None,'','None']:
                    raise NameError('TitleID')

        except KeyError:
            print('KeyError:',title)
            continue
        except IndexError:
            print('IndexError:',title)
            continue
        except NameError:
            print('TitleID',title)
            continue     

        # Get episodes
        r = requests.get('https://api.themoviedb.org/3/tv/'+title_id+'?api_key='+api_key)
        parsed = json.loads(r.text)
        #pretty_data = json.dumps(parsed, indent=4, sort_keys=True)
        #print(pretty_data)
        
        try:
            df.loc[title, 'IMGID'] = parsed['poster_path']
            df.loc[title, 'Episodes'] = parsed['number_of_episodes']
            df.loc[title, 'First Air Date'] = parsed['first_air_date']
            df.loc[title, 'Last Air Date'] = parsed['last_air_date']
        except KeyError:
            print('KeyError:',title)
            continue
        except IndexError:
            print('IndexError:',title)
            continue
        except NameError:
            print('TitleID',title)
            continue     

0 Neon Genesis Evangelion
50 Death Note
100 Space☆Dandy
150 Assassination Classroom
200 Key the Metal Idol
TitleID Black Jack (OVA)
TitleID Dirty Pair (1985)
250 JoJo's Bizarre Adventure (1993)
TitleID Tetsujin 28-Gou (2004)
TitleID Manga Children's Library
300 Re:Zero
TitleID To My Brother...
TitleID Ratio of Forest
TitleID The Diary of Tortov Roddle
350 Occultic;Nine
TitleID GARTO: THE ANIMATION
TitleID Neighborhood Stories
TitleID The Crayon Kingdom of Dreams
400 Haré+Guu
TitleID Urotsukidoji
450 Beck


In [14]:
#df=df.astype({'Episodes':'int','Votes':'int','Rank':'int','Esoteric Rank':'int','Episodes':'int','AniListID':'int','TMDbID':'int'})
df.head()

,Rank,Votes,Score,Esoteric Rank,Esoteric Score,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Neon Genesis Evangelion,1,56,1238.1,148,17.9,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,
Serial Experiments Lain,2,46,987.0,180,12.5,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/oQOhIIZWKZpeQS6uZejS9rhaHWO.jpg,
Cowboy Bebop,3,43,870.0,185,12.2,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,
FLCL,4,33,700.0,220,8.6,Original,18,Gainax,2000-04-26,2018-10-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,
Revolutionary Girl Utena,5,28,658.0,186,12.0,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,


In [15]:
# Numpy types not JSON serializable => convert np.int64 to native Python int by casting to 'object'
# b=vdf.astype('object')

# Or: convert to string matrix to push NaNs as empty string '' to sheet
s = vdf.fillna('')
s = s.astype(str)

individual_votes = [s[column].tolist() for column in s]

# Push voters to sheet

voters = vdf.columns.tolist()
POINTS_RANGE = 'Data!'+start+'1'
POINTS_VALUES = [voters]
push_sheet_data(SCOPES,SPREADSHEET_ID,POINTS_RANGE,POINTS_VALUES,'ROWS')

# Push individual votes to sheet

POINTS_RANGE = 'Data!'+start+'2'
POINTS_VALUES = individual_votes
push_sheet_data(SCOPES,SPREADSHEET_ID,POINTS_RANGE,POINTS_VALUES,'COLUMNS')

# Push title data to sheet

def f(col):
    c = col.tolist()
    c.insert(0,col.name)
    return c


titles = list(titlesdf)
titles.insert(0,'Title')
rank = f(df['Rank'])
votes = f(df['Votes'])
score = f(df['Score'])
esoteric_rank = f(df['Esoteric Rank'])
esoteric_score = f(df['Esoteric Score'])
source = f(df['Source'])
episodes = f(df['Episodes'])
studio = f(df['Studio'])
genres = f(df['Genres'])
firstair = f(df['First Air Date'])
lastair = f(df['Last Air Date'])
alttitle = f(df['AltTitle'])
anilist = f(df['AniListID'])
tmdb = f(df['TMDbID'])
img = f(df['IMGID'])

POINTS_RANGE = 'Data!A1'
POINTS_VALUES = [titles,rank,votes,score,esoteric_rank,esoteric_score,source,episodes,studio,firstair,lastair,genres,anilist,tmdb,img,alttitle]
push_sheet_data(SCOPES,SPREADSHEET_ID,POINTS_RANGE,POINTS_VALUES,'COLUMNS')

77 cells updated.
36806 cells updated.
7635 cells updated.
